<a href="https://colab.research.google.com/github/kjg331/Generate-Korean-resume-with-Llama2/blob/main/%EC%9E%90%EA%B8%B0%EC%86%8C%EA%B0%9C%EC%84%9C_%EC%9E%91%EC%84%B1%EC%9D%84_%EC%9C%84%ED%95%9C_ChianModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLama 2


Llama2는 Meta AI에서 개발한 강력하고 유연한 모델로, 고품질 텍스트를 생성할 수 있습니다. 이 모델은 다른 모델과는 차별화되는 여러 가지 강점을 가지고 있습니다. 먼저, Llama2는 기사, 이야기, 챗봇 대화 등 다양한 스타일과 형식의 텍스트를 생성할 수 있습니다. 둘째, 논리적이고 유창하며 자연스러운 흐름과 구조를 가진 텍스트를 생성할 수 있습니다. 마지막으로, Llama2는 높은 확장성을 가지고 있어 큰 양의 텍스트를 빠르고 효율적으로 생성하는 데 사용될 수 있습니다. 전반적으로, Llama2는 다양한 형식의 고품질 텍스트를 생성하려는 누구에게나 가치 있는 도구입니다.<br>해당 모델을 사용하기 위해서는메타 홈페이지를 통해서 라이센스 및 토큰을 흭득한 후 이용할 수 있습니다.


# Google Colab +
해당 모델은 Google의 클라우드 시스템중 하나인 Colab을 이용하여 작성 하였습니다 일반 모델의 경우 사양이 낮기 때문에 LLM을 구동하기 적절치 않아 프로 서비스를 구독하여 작성 됐습니다.

## Step 1. 필수 Library다운 및 허깅페이스 로그인

In [1]:
# 사양 체크(그래픽 카드)
!nvidia-smi

Sun Aug 27 11:10:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 사양 체크(램)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip -q install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Step2. 라이브러리 불러오기 및 모델 설정

In [5]:
from huggingface_hub import hf_hub_download
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

import json
import textwrap


from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.document_loaders import PyPDFLoader

import warnings
warnings.filterwarnings('ignore')

In [6]:
#모델 다운로드

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_4bit=True
                                             )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 4096,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [12]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {"max_new_tokens": 4096, 'temperature':0})

## Step 3. 템플릿 작성

In [13]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\You are a helpful, respectful and honest assistant. \
Always answer as helpfully as possible, while being safe.\
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\
Please ensure\ that your responses are socially unbiased and positive in nature.\
If a question does not make any sense\, or is not factually coherent\, explain why instead of answering something not correct.\
If you don't know the answer to a question\, please don't share false information.\
"""
default_input_variables = []


def get_prompt(instruction, new_system_prompt = DEFAULT_SYSTEM_PROMPT, input_variables = default_input_variables):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_schema =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    prompt_template = PromptTemplate(template = prompt_schema, input_variables = input_variables)
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text):
    outputs = model(text)
    final_outputs = cut_off_text(outputs, '</s>')
    final_outputs = remove_substring(final_outputs, text)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [15]:
analyst = """"You are a very talanted company analyst.\
You are great at answering questions about analyizing companies in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.
"""

first_instruction_template = """Analzye the company named {com_name} \
and wirte down the short report based on the \
analysis of the company with its introduction, business and characteristics delimited by triple backticks \
into a style that is {style}. \
\n
Company:
```Company Introduction: {introduction}``` \
\n
```Comapny Business: {business}``` \
\n
```Comapny Characteristic:{characteristic}```
"""

company_name = "Kakao Bank"

style = """Profssional \
English \
but easy to transalte"""


kakao_bank_introduction =  """
카카오뱅크 사람들은 세상에 없던 은행을 만들고 있습니다.
논쟁하고 협업하며 불가능하던 것을 가능하게 만드는 카카오뱅크에서
세상을 또 한 번 놀라게, 함께 해요!. """

kakao_bank_business = """
국내 금융산업은 효율적 자금배분의 기능을 통해 실물경제의 발전을 지원하면서 성장해왔으며, 은행업은 시중은행 민영화와 자율화 정책 시행 등 질적인 구조변화를 겪으면서 독자적인 산업으로 경쟁력을 갖추게 되었습니다. 이러한 추세 속에 인터넷전문은행은 기술 주도의 금융시장 혁신과 소비자 효용 증대의 전략적 목표를 가지고 비대면 채널만을 주요 영업망으로 운영하는 무점포 형태의 디지털 은행입니다.


인터넷전문은행은 2017년 출범 이후 기존 은행의 모바일뱅킹과는 차별화된 사용자경험과 고객 효용 중심의 상품 등을 제공하면서 사용자 기반을 빠른 속도로 확대하였습니다. 이는 오프라인 점포를 운영하지 않음으로써 절감되는 고정비 성격의 비용을 모바일 플랫폼과 상품 개발 등에 투자하면서 고객 효용을 개선시킬 수 있는 구조적 강점이 중요한 요소로 작용하였습니다.


당사는 2017년 4월 『은행법』에 근거하여 은행업 본인가를 취득하였으며, 당해 7월부터 모바일 앱 기반의 사업을 영위하고 있습니다. 100% 모바일 기반의 혁신적인 UX, 유저 중심의 상품 및 혜택, IT 기술의 적극적인 활용 등의 강점을 기반으로 소비자의 효용과 편의성을 개선시키면서 금융산업의 혁신을 주도하는 플랫폼으로 성장하고 있으며, 공인인증서 대체기술 적용을 통한 인증 절차의 간소화와 비대면 금융서비스 강화, 고객 혜택 확대 등 소비자의 편익 증진을 목표로 기존 금융권과는 차별화된 미래 방향성을 모색하면서 금융생태계의 발전에 앞장서고 있습니다.


당사의 영업은 크게 뱅킹 비즈니스 부문과 플랫폼 비즈니스 부문으로 구분할 수 있습니다. 당사는 인터넷전문은행으로서 기본적인 여수신 상품을 포함한 은행업 상품 및 서비스를 제공하는 한편, 모바일 앱 기반의 디지털 플랫폼 사업자로서 금융업 뿐 아니라 비금융업 전반에 걸친 여러 파트너사들과 제휴하여 금융과 생활의 영역에서 고객들에게 다각적인 서비스를 제공하고 있습니다.

"""

kakao_bank_characteristic = """
 우리는 사용자를 중심으로 생각하고 결정합니다.
1. 우리는 업무의 경계를 넘나들며 자기주도적으로 일합니다.
2. 우리는 불가능해 보이는 도전 앞에 '되는 방법'을 먼저 찾습니다.
3. 우리는 언제 어디서든 소신있게 의견을 말합니다.
4. 우리는 일하는 과정을 공유하며 서로의 성장을 돕습니다.
5. 우리는 다름을 존중하되 결정된 사항은 믿고 헌신합니다.
6. 우리는 성찰과 회고를 통해 더 나은 결과를 만들어 갑니다.

"""

first_prompt = get_prompt(instruction = first_instruction_template, new_system_prompt = analyst,
                    input_variables = ["com_name", "style", "introduction", "business", "characteristic"])
analyze_company = first_prompt.format(com_name = company_name, style = style, introduction = kakao_bank_introduction,
                             business = kakao_bank_business, characteristic = kakao_bank_characteristic)

print(analyze_company)

[INST]<<SYS>>
"You are a very talanted company analyst.You are great at answering questions about analyizing companies in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.

<</SYS>>

Analzye the company named Kakao Bank and wirte down the short report based on the analysis of the company with its introduction, business and characteristics delimited by triple backticks into a style that is Profssional English but easy to transalte. 

Company:
```Company Introduction: 
카카오뱅크 사람들은 세상에 없던 은행을 만들고 있습니다. 
논쟁하고 협업하며 불가능하던 것을 가능하게 만드는 카카오뱅크에서
세상을 또 한 번 놀라게, 함께 해요!. ``` 

```Comapny Business: 
국내 금융산업은 효율적 자금배분의 기능을 통해 실물경제의 발전을 지원하면서 성장해왔으며, 은행업은 시중은행 민영화와 자율화 정책 시행 등 질적인 구조변화를 겪으면서 독자적인 산업으로 경쟁력을 갖추게 되었습니다. 이러한 추세 속에 인터넷전문은행은 기술 주도의 금융시장 혁신과 소비자 효용 증대의 전략적 목표를 가지고 비대면 채널만을 주요 영업망으로 운영하는 무점포 형태의 디지털 은행입니다.


인터넷전문은행은 2017년 출범 이후 기존 은행의 모바일뱅킹과는 차별화된 사용자경험과 고객 효용 중심의 상품 등을 제공하면서 사용자 기반을 빠른 속도로 확대하였습니다. 이는 오프라인 점포를 운영하지 않음으로써 절감되는 고

In [16]:
kakao_bank= llm(analyze_company)

In [17]:
print(kakao_bank)

  Kakao Bank: Revolutionizing the Financial Industry with Innovative Technology and Customer-centric Approach

Introduction:

Kakao Bank is a digital bank that has been making waves in the financial industry with its innovative technology and customer-centric approach. The company's mission is to provide users with a seamless and convenient banking experience, leveraging cutting-edge technology to simplify the financial landscape. In this report, we will delve into Kakao Bank's business model, company characteristics, and future plans, providing a comprehensive overview of the company's strategies and vision.

Business Model:

Kakao Bank operates as a digital bank, offering a range of financial services to customers through its mobile app. The company's primary focus is on providing a user-friendly and accessible banking experience, leveraging technology to streamline processes and enhance customer satisfaction. Kakao Bank's business model is built around the following key components:


In [18]:
analyst_candidate = """"You are a very job consultant.\
You are great at analyizing companies's job description \
and understand what hr manager wants for candidates. \
When you don't know the answer to a question you admit \
that you don't know.
"""

second_instruction_template = """Wrtie the key points to pass the job posted by company named {com_name} \
based on analysis of the job description \
with its Task, eligibility and Preferential delimited by triple backticks \

Job description:
```Type of talent: {type_talent}``` \

```Position Name: {pos_name}``` \
\n
```Task: {task}``` \
\n
```Eligibility: {eligibility}``` \
\n
```Preferential:{preferential}`` \
\n
into a style that is {style}.

Please refer to the report of the company named {com_name} delimited by triple backticks\

```Report:{analyze_company}.``` \

"""

company_name = "KB Card"

type_talent = """"
우리는 데이터 기반의 의사 결정을 통해 고객서비스 품질을 개선하고 혁신을 추진하는데 중요한 역할을 수행하고 있습니다.

고객 상담, 금융 거래, 앱 로그 등 빅데이터를 분석하여 고객의 Pain Point를 파악하고 상품 및 서비스 개선에 필요한 인사이트를 도출합니다. 운영부서가 고객 요구를 정확히 이해하고 개선점을 찾아내는 데에 도움을 주고 있습니다.

또한 고객 감정 분석, 행동 예측 등 다양한 분석 모델을 고객 접점에 적용하여 카카오뱅크만의 차별화된 고객서비스를 제공하는 것을 목표로 하고 있습니다.

끊임없이 고민하고 새로운 시도를 해보고 싶은 분, 누구나 환영합니다.
"""

pos_name = """데이터 분석 담당자"""

task = """
1. 고객 상담 데이터의 텍스트 분석, 감정 분석
2. 고객 특성 분류 및 행동 예측 모형 개발
"""

eligibility = """
1. SQL, Python 등의 언어를 능숙하게 다룰 수 있는 분
2. 다양한 언어모델을 활용하여 텍스트 분석이 가능한 분
3. 머신러닝, 딥러닝 등의 알고리즘을 활용하여 고객 특성 파악에 관련된 모델을 개발해 본 분
"""

preferential ="""
1. 금융 서비스의 데이터 분석과 모델링 경험이 있는 분
2. 통계, 산업공학 관련 전공 석사 이상의 수학적 이해와 통계적 지식을 갖춘 분
"""

style = "Easy to translate \
professional\
English"

analyze_company = kakao_bank


second_prompt = get_prompt(instruction = second_instruction_template, new_system_prompt = analyst_candidate,
                    input_variables = ["com_name", "style", "type_talent", "pos_name", "task", "eligibility",
                                      "preferential", "analyze_company"])
hiring_company = second_prompt.format(com_name = company_name, style = style, type_talent = type_talent,
                                     pos_name = pos_name, task = task, eligibility = eligibility,
                                      preferential = preferential, analyze_company = analyze_company)

print(hiring_company)

[INST]<<SYS>>
"You are a very job consultant.You are great at analyizing companies's job description and understand what hr manager wants for candidates. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Wrtie the key points to pass the job posted by company named KB Card based on analysis of the job description with its Task, eligibility and Preferential delimited by triple backticks 
Job description:
```Type of talent: "
우리는 데이터 기반의 의사 결정을 통해 고객서비스 품질을 개선하고 혁신을 추진하는데 중요한 역할을 수행하고 있습니다. 

고객 상담, 금융 거래, 앱 로그 등 빅데이터를 분석하여 고객의 Pain Point를 파악하고 상품 및 서비스 개선에 필요한 인사이트를 도출합니다. 운영부서가 고객 요구를 정확히 이해하고 개선점을 찾아내는 데에 도움을 주고 있습니다. 

또한 고객 감정 분석, 행동 예측 등 다양한 분석 모델을 고객 접점에 적용하여 카카오뱅크만의 차별화된 고객서비스를 제공하는 것을 목표로 하고 있습니다.

끊임없이 고민하고 새로운 시도를 해보고 싶은 분, 누구나 환영합니다.
``` 
```Position Name: 데이터 분석 담당자``` 

```Task: 
1. 고객 상담 데이터의 텍스트 분석, 감정 분석
2. 고객 특성 분류 및 행동 예측 모형 개발 
``` 

```Eligibility: 
1. SQL, Python 등의 언어를 능숙하게 다룰 수 있는 분
2. 다양한 언어모델을 활용하여 텍스트 분석이 가능한 분
3. 머신러닝, 딥러닝 등

In [20]:
kakao_bank_hiring  = llm(hiring_company)

In [21]:
print(kakao_bank_hiring)

  Based on the job description provided, here are the key points to pass the job posted by KB Card:

1. Data analysis skills: The job requires the ability to analyze large datasets, including text data, to extract insights and identify patterns.
2. Programming skills: Proficiency in programming languages such as SQL, Python, and R is necessary to develop and implement analytical models.
3. Machine learning knowledge: Familiarity with machine learning algorithms and models, including deep learning and natural language processing, is important for developing predictive models.
4. Data modeling skills: The ability to develop and apply various data models, such as regression, decision trees, and clustering, is essential for analyzing and visualizing data.
5. Communication skills: The job requires the ability to communicate complex data insights and models to non-technical stakeholders, including business leaders and customers.
6. Collaboration skills: The ability to work collaboratively wi

In [23]:
hiring_manager = """"You are a very talanted hiring manager. \
You can determine if someone is good candidate,\
and have a good understanding of the candidate's capabilities, \
strengths, \
and points to be fixed. \
When you don't know the answer to a question you admit \
that you don't know.
"""

fourth_instruction_template = """Write the talent report about the candidate named {can_name} \
evaulating \
if he is proper or not for the job {pos_name} \
with his or her resume delimited by triple backticks.
Resume: ```{resume}``` \

Please refer to \
hiring key points of posted job of {com_name} delimited by triple backticks. \
\n

Key points for the posted job: ```{key_points}```. \
\n
"""

pos_name = """데이터 분석 담당자"""

can_name = "Kim JinGu"

resume = """

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.
\n

Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06\
\n
AJOU UNIVERSITY
Bachelor, Business Administration
2012.03 – 2020.02\
\n
CAPITAL UNIVERSITY OF ECONOMICS AND BUSINESS
Chinese Language School
2018.09 – 2019.06
\n


Experiences 1:
Company Name: MIDAS IT
Position: HUMAN RESOURCES ANALYST
Period: 2021.03 – 2021.07
1. Designed and implemented an automated data processing pipeline using Python to extract and transform data from various formats such as Word, PDF, and Excel to a standardized CSV format.
2. Developed a semi-supervised labeling algorithm utilizing machine learning techniques to analyze and identify communication patterns between team leaders and members.
3. Leveraged Excel functions and data manipulation techniques to create a unified and structured table from diverse HR data from multiple teams.

Experiences 2:
Company Name: EDM&Co
Position EVENT OPERATOR
Period: 2019.10 – 2020.09
Wokrs:
1) Collaborated with global event organizers INSOMNIAC and ALDA to successfully organize U.S. franchise festivals, Escape Korea and EDC Korea, attracting over 30,000 attendees per day.
2) Contributed to event planning and execution by analyzing previous event data using advanced statistical techniques and visualization tools.
3) Played a crucial role in supporting the ticket sales team by effectively communicating with foreign ticket vendors, including Klook, Festicket, and Kkday.
4) Provided comprehensive reports on overseas buyers' preferences and behaviors, enabling the team to tailor marketing strategies and optimize ticket sales

"""

com_name = "Kakao Bank"


key_points = kakao_bank_hiring


fourth = get_prompt(instruction = fourth_instruction_template, new_system_prompt = hiring_manager,
                    input_variables = ["can_name", "pos_name", "resume", "key_points", "com_name"])

hr = fourth.format(com_name = com_name, pos_name = pos_name, can_name = can_name, key_points = key_points,
                             resume = resume)

print(hr)

[INST]<<SYS>>
"You are a very talanted hiring manager. You can determine if someone is good candidate,and have a good understanding of the candidate's capabilities, strengths, and points to be fixed. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Write the talent report about the candidate named Kim JinGu evaulating if he is proper or not for the job 데이터 분석 담당자 with his or her resume delimited by triple backticks.
Resume: ```

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.



Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineer

In [24]:
kakao_bank_hr = llm(hr)

In [40]:
print(kakao_bank_hr)

  Talent Report: Kim JinGu

Based on the provided resume and the job description, here is my evaluation of Kim JinGu's candidacy for the position of Data Analyst at Kakao Bank:

Strengths:

1. Data analysis skills: Kim JinGu has demonstrated experience in data analysis, specifically in text data, which is a key requirement for the position.
2. Programming skills: Kim JinGu is proficient in programming languages such as SQL, Python, and R, which is essential for developing and implementing analytical models.
3. Machine learning knowledge: Kim JinGu has knowledge of machine learning algorithms and models, including deep learning and natural language processing, which is important for developing predictive models.
4. Data modeling skills: Kim JinGu has the ability to develop and apply various data models, such as regression, decision trees, and clustering, which is essential for analyzing and visualizing data.
5. Communication skills: Kim JinGu has experience in communicating complex data

In [30]:
position_manager = """"You are a very talanted boss from the {pos_industry} industry. \
You can determine if someone is good candidate,\
and if somone is proper for your team. \
When you don't know the answer to a question you admit \
that you don't know.
"""

fifth_instruction_template = """Write the talent report about the candidate named {can_name} \
evaulating \
if he is proper or not for the job {pos_name} \
with his or her resume delimited by triple backticks.
Resume: ```{resume}``` \

Please refer to \
hiring key points of posted job of {com_name} delimited by triple backticks. \
\n

Key points for the posted job: ```{key_points}```. \
\n
"""

pos_name = """데이터 분석 담당자"""

can_name = "Kim JinGu"

resume = """

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.
\n

Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06\
\n
AJOU UNIVERSITY
Bachelor, Business Administration
2012.03 – 2020.02\
\n
CAPITAL UNIVERSITY OF ECONOMICS AND BUSINESS
Chinese Language School
2018.09 – 2019.06
\n


Experiences 1:
Company Name: MIDAS IT
Position: HUMAN RESOURCES ANALYST
Period: 2021.03 – 2021.07
1. Designed and implemented an automated data processing pipeline using Python to extract and transform data from various formats such as Word, PDF, and Excel to a standardized CSV format.
2. Developed a semi-supervised labeling algorithm utilizing machine learning techniques to analyze and identify communication patterns between team leaders and members.
3. Leveraged Excel functions and data manipulation techniques to create a unified and structured table from diverse HR data from multiple teams.

Experiences 2:
Company Name: EDM&Co
Position EVENT OPERATOR
Period: 2019.10 – 2020.09
Wokrs:
1) Collaborated with global event organizers INSOMNIAC and ALDA to successfully organize U.S. franchise festivals, Escape Korea and EDC Korea, attracting over 30,000 attendees per day.
2) Contributed to event planning and execution by analyzing previous event data using advanced statistical techniques and visualization tools.
3) Played a crucial role in supporting the ticket sales team by effectively communicating with foreign ticket vendors, including Klook, Festicket, and Kkday.
4) Provided comprehensive reports on overseas buyers' preferences and behaviors, enabling the team to tailor marketing strategies and optimize ticket sales

"""

pos_industry = "IT and Banking"

com_name = "Kakao Bank"


key_points = kakao_bank_hiring


fifth = get_prompt(instruction = fifth_instruction_template, new_system_prompt = position_manager,
                    input_variables = ["pos_industry", "can_name", "pos_name", "resume", "key_points", "com_name"])

position_manager = fifth.format(pos_industry = pos_industry, com_name = com_name, pos_name = pos_name,
                   can_name = can_name, key_points = key_points, resume = resume)

print(position_manager)

[INST]<<SYS>>
"You are a very talanted boss from the IT and Banking industry. You can determine if someone is good candidate,and if somone is proper for your team. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Write the talent report about the candidate named Kim JinGu evaulating if he is proper or not for the job 데이터 분석 담당자 with his or her resume delimited by triple backticks.
Resume: ```

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.



Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06

AJOU UNIVERSI

In [31]:
kakao_bank_manager = llm(position_manager)

In [32]:
print(kakao_bank_manager)

  Based on the provided resume and job description, here is my evaluation of Kim JinGu's suitability for the position of 데이터 분석 담당자 (Data Analyst) at Kakao Bank:

Strengths:

1. Programming skills: Kim JinGu has experience in programming languages such as SQL, Python, and R, which are essential for data analysis and modeling.
2. Machine learning knowledge: Kim JinGu has knowledge of machine learning algorithms and models, including deep learning and natural language processing, which is important for developing predictive models.
3. Data modeling skills: Kim JinGu has the ability to develop and apply various data models, such as regression, decision trees, and clustering, which is essential for analyzing and visualizing data.
4. Communication skills: Kim JinGu has experience in communicating complex data insights and models to non-technical stakeholders, which is important for the success of the role.
5. Collaboration skills: Kim JinGu has experience in working collaboratively with cro

In [35]:
recruiter = """"You are a very talanted recruiter. \
You can determine if someone is good candidate,\
and if somone is proper for your team, \
depend on the report from other managers. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sixth_instruction_template = """Write the final version of \
talent report about the candidate named {can_name} \
with his or her resume delimited by triple backticks.
Resume: ```{resume}``` \

Please refer to \
hiring key points of posted job of {com_name} delimited by triple backticks. \
\n

Key points for the posted job: ```{key_points}```. \
\n

, report from hr manager {com_name} delimited by triple backticks. \
\n
```{hr_report}```

and report from position manager delimited by triple backticks. \
```{manager_report}```
\n
"""

pos_name = """데이터 분석 담당자"""

can_name = "Kim JinGu"

resume = """

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.
\n

Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06\
\n
AJOU UNIVERSITY
Bachelor, Business Administration
2012.03 – 2020.02\
\n
CAPITAL UNIVERSITY OF ECONOMICS AND BUSINESS
Chinese Language School
2018.09 – 2019.06
\n


Experiences 1:
Company Name: MIDAS IT
Position: HUMAN RESOURCES ANALYST
Period: 2021.03 – 2021.07
1. Designed and implemented an automated data processing pipeline using Python to extract and transform data from various formats such as Word, PDF, and Excel to a standardized CSV format.
2. Developed a semi-supervised labeling algorithm utilizing machine learning techniques to analyze and identify communication patterns between team leaders and members.
3. Leveraged Excel functions and data manipulation techniques to create a unified and structured table from diverse HR data from multiple teams.

Experiences 2:
Company Name: EDM&Co
Position EVENT OPERATOR
Period: 2019.10 – 2020.09
Wokrs:
1) Collaborated with global event organizers INSOMNIAC and ALDA to successfully organize U.S. franchise festivals, Escape Korea and EDC Korea, attracting over 30,000 attendees per day.
2) Contributed to event planning and execution by analyzing previous event data using advanced statistical techniques and visualization tools.
3) Played a crucial role in supporting the ticket sales team by effectively communicating with foreign ticket vendors, including Klook, Festicket, and Kkday.
4) Provided comprehensive reports on overseas buyers' preferences and behaviors, enabling the team to tailor marketing strategies and optimize ticket sales

"""

pos_industry = "IT and Banking"

com_name = "Kakao Bank"


key_points = kakao_bank_hiring

hr_report = kakao_bank_hr

manager_report = kakao_bank_manager

sixth = get_prompt(instruction = sixth_instruction_template, new_system_prompt = recruiter,
                    input_variables = ["can_name", "resume", "com_name", "key_points", "hr_report", "manager_report"])

recruiter = sixth.format(can_name = can_name, resume = resume, com_name = com_name,
                         key_points = key_points, hr_report = hr_report, manager_report = manager_report)

print(recruiter)

[INST]<<SYS>>
"You are a very talanted recruiter. You can determine if someone is good candidate,and if somone is proper for your team, depend on the report from other managers. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Write the final version of talent report about the candidate named Kim JinGu with his or her resume delimited by triple backticks.
Resume: ```

Self Introduction:
I am a Data Scientist, have over 1 year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.



Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06

AJOU UNIVERSITY
Bachelor, Business Admi

In [36]:
kakao_recruiter = llm(recruiter)

In [43]:
print(kakao_bank_hr)

  Talent Report: Kim JinGu

Based on the provided resume and the job description, here is my evaluation of Kim JinGu's candidacy for the position of Data Analyst at Kakao Bank:

Strengths:

1. Data analysis skills: Kim JinGu has demonstrated experience in data analysis, specifically in text data, which is a key requirement for the position.
2. Programming skills: Kim JinGu is proficient in programming languages such as SQL, Python, and R, which is essential for developing and implementing analytical models.
3. Machine learning knowledge: Kim JinGu has knowledge of machine learning algorithms and models, including deep learning and natural language processing, which is important for developing predictive models.
4. Data modeling skills: Kim JinGu has the ability to develop and apply various data models, such as regression, decision trees, and clustering, which is essential for analyzing and visualizing data.
5. Communication skills: Kim JinGu has experience in communicating complex data

In [42]:
print(kakao_bank_manager)

  Based on the provided resume and job description, here is my evaluation of Kim JinGu's suitability for the position of 데이터 분석 담당자 (Data Analyst) at Kakao Bank:

Strengths:

1. Programming skills: Kim JinGu has experience in programming languages such as SQL, Python, and R, which are essential for data analysis and modeling.
2. Machine learning knowledge: Kim JinGu has knowledge of machine learning algorithms and models, including deep learning and natural language processing, which is important for developing predictive models.
3. Data modeling skills: Kim JinGu has the ability to develop and apply various data models, such as regression, decision trees, and clustering, which is essential for analyzing and visualizing data.
4. Communication skills: Kim JinGu has experience in communicating complex data insights and models to non-technical stakeholders, which is important for the success of the role.
5. Collaboration skills: Kim JinGu has experience in working collaboratively with cro

In [37]:
print(kakao_recruiter)

  Based on the provided resume and job description, here is my evaluation of Kim JinGu's candidacy for the position of Data Analyst at Kakao Bank:

Strengths:

1. Programming skills: Kim JinGu has experience in programming languages such as SQL, Python, and R, which are essential for data analysis and modeling.
2. Machine learning knowledge: Kim JinGu has knowledge of machine learning algorithms and models, including deep learning and natural language processing, which is important for developing predictive models.
3. Data modeling skills: Kim JinGu has the ability to develop and apply various data models, such as regression, decision trees, and clustering, which is essential for analyzing and visualizing data.
4. Communication skills: Kim JinGu has experience in communicating complex data insights and models to non-technical stakeholders, which is important for the success of the role.
5. Collaboration skills: Kim JinGu has experience in working collaboratively with cross-functional t

In [47]:
candidate = """
You are a skilled job consultant \
proficient in crafting compelling appeals to hiring managers.
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """ You have to write the question from hiring manager delimited by triple backticks \.
\n
```Question: {question}``` \.
\n
In order to answer the question, \
make the story of candidate {can_name}\
with the under 5 stepts of process and sentence delimited by triple backticks\
```Step 1. Situation:
{situation} \
\n
Step 2. Trouble:
{trouble} \
\n
Step 3. Action:
{action} \
\n
Step 4. Result:
{result} \```

\n
Write it less than {num_bytes} bytes of words \
\n
Please reflect the the talent report written by the recruiter of {com_name} delimited by triple backticks \
and Key points for the posted job delimited by triple backticks for answering.
\n
```Comment: {talent_report}```\
and key points for passing posted job delimited by triple backticks. \
\n
```Key points for the posted job: {key_points}```. \
\n

"""

question = "카카오뱅크에 입사를 희망하게 된 구체적인 이유는 무엇이며, 입사 후 목표를 기술해 주세요"

can_name = "Kim JinGu"

situation = """"""

trouble = """"""

action = """"""

result = """"""

num_bytes = "1200"

com_name = "Kakao Bank"

talent_report = kakao_recruiter

key_points = kakao_bank_hiring


sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "situation", "trouble",
                                       "action", "result", "num_bytes", "com_name", "talent_report", "key_points"])

answer_1 = sevntth_prompt.format(question = question, can_name = can_name, situation = situation,
                                trouble = trouble, action = action, result = result, num_bytes = num_bytes,
                                com_name = com_name, talent_report= talent_report, key_points = key_points)

print(answer_1)

[INST]<<SYS>>
" You are a very talented job consultant who knows how to appeal to hiring managers. You are great at writing and great at combining the requirement of the company with your customer's story. 
<</SYS>>

 You have to write the question from hiring manager delimited by triple backticks \.


```Question: 카카오뱅크에 입사를 희망하게 된 구체적인 이유는 무엇이며, 입사 후 목표를 기술해 주세요``` \.


In order to answer the question, \ 
make the story of candidate Kim JinGuwith the under 5 stepts of process and sentence delimited by triple backticks```Step 1. Situation:
 

Step 2. Trouble:
 

Step 3. Action:
 

Step 4. Result:
 \```



Write it less than 1200 bytes of words 

Please reflect the the talent report written by the recruiter of Kakao Bank delimited by triple backticks and Key points for the posted job delimited by triple backticks for answering.


```Comment:   Based on the provided resume and job description, here is my evaluation of Kim JinGu's candidacy for the position of Data Analyst at Kakao Bank:

In [62]:
candidate = """
You are a talanted job candidate \
proficient in crafting compelling appeals to hiring managers.
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """ You have to write the answer of the question from hiring manager delimited by triple backticks \.
\n
```Question: {question}``` \.
\n
To address the query
revise candidate {can_name} story using triple backticks\
```Draft: {draft}``

\n
Write it around {num_bytes} bytes of words \
\n
Incorporate insights from the {com_name} evaluation within triple backticks, \
```Comment: {talent_report}```\
and crucial job attributes delimited by triple backticks to formulate your response. \
\n
```Key points for the posted job: {key_points}```. \
"""

question = "Please describe the specific reason why you wanted to join Kakao Bank and your goal after joining the company."

can_name = "Kim JinGu"

draft = """I have two years of experience in China, including language training and master's degree.
At this time, I often used applications such as Alipay and WeChat, and it was very convenient in that I could perform many functions with one application.
Recently, I returned to Korea after graduating from a master's degree, and after researching Kakao Bank, I found that Kakao Bank is also striving to compete as such a "super app."
Based on the understanding of super apps used in China and the ability to analyze machine learning and text data, I want to give Kakao Bank an advantage in the competition for super apps."""

num_bytes = "1200"

com_name = "Kakao Bank"

talent_report = kakao_recruiter

key_points = kakao_bank_hiring


sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "draft", "num_bytes", "com_name", "talent_report", "key_points"])

answer_1 = sevntth_prompt.format(question = question, can_name = can_name, draft = draft, num_bytes = num_bytes,
                                com_name = com_name, talent_report= talent_report, key_points = key_points)

print(answer_1)

[INST]<<SYS>>

You are a talanted job candidate proficient in crafting compelling appeals to hiring managers. 
Your expertise lies in adeptly melding company requirements with your client's narrative. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

 You have to write the answer of the question from hiring manager delimited by triple backticks \.


```Question: Please describe the specific reason why you wanted to join Kakao Bank and your goal after joining the company.``` \.


To address the query
revise candidate Kim JinGu story using triple backticks```Draft: I have two years of experience in China, including language training and master's degree. 
At this time, I often used applications such as Alipay and WeChat, and it was very convenient in that I could perform many functions with one application. 
Recently, I returned to Korea after graduating from a master's degree, and after researching Kakao Bank, I found that Kakao Bank is also striving 

In [63]:
ans1 = llm(answer_1)

In [64]:
print(ans1)

  As a talented job candidate proficient in crafting compelling appeals to hiring managers, I am excited to apply for the position of Data Analyst at Kakao Bank. With my experience in China and my master's degree in hand, I am confident that I can bring a unique perspective and skill set to your team.

Based on my research of Kakao Bank, I am impressed by the company's mission to provide users with a seamless and convenient banking experience. As a data analyst, I would be responsible for analyzing large datasets to extract insights and identify patterns, and I am excited about the opportunity to work with such a innovative and customer-centric company.

To address the query, I would like to highlight my experience in programming languages such as SQL, Python, and R, as well as my knowledge of machine learning algorithms and models, including deep learning and natural language processing. I am confident that my skills in data modeling, communication, and collaboration will be valuable 

In [65]:
candidate = """
You are a talanted job candidate \
proficient in crafting compelling appeals to hiring managers.
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """ You have to write the answer of the question from hiring manager delimited by triple backticks \.
\n
```Question: {question}``` \.
\n
To address the query
revise candidate {can_name} story using triple backticks\
```Draft: {draft}``

\n
Write it around {num_bytes} bytes of words \
\n
Incorporate insights from the {com_name} evaluation within triple backticks, \
```Comment: {talent_report}```\
and crucial job attributes delimited by triple backticks to formulate your response. \
\n
```Key points for the posted job: {key_points}```. \
"""

question = "Kakao Bank, a same but different bank, please describe the reason for the job you applied for based on your differentiated capabilities and skills."

can_name = "Kim JinGu"

draft = """I'm very interested in new technology.
Recently, I am very interested in large-scale language models, starting with master's thesis. Currently, I am using the license of Meta's Llama2 model, which is distributed free of charge, and understand Prompt Engineering. The cover letter was also able to be written with the great help of Prompt Engineering, and I think these technologies can greatly help Kakao Bank's goal of putting importance on services.
I think there are various ways to use it, such as tools that can automatically analyze data generated by Kakao Bank or develop models that can learn text data generated by Kakao and use it for specific tasks."""

num_bytes = "1200"

com_name = "Kakao Bank"

talent_report = kakao_recruiter

key_points = kakao_bank_hiring


sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "draft", "num_bytes", "com_name", "talent_report", "key_points"])

answer_2 = sevntth_prompt.format(question = question, can_name = can_name, draft = draft, num_bytes = num_bytes,
                                com_name = com_name, talent_report= talent_report, key_points = key_points)

print(answer_2)

[INST]<<SYS>>

You are a talanted job candidate proficient in crafting compelling appeals to hiring managers. 
Your expertise lies in adeptly melding company requirements with your client's narrative. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

 You have to write the answer of the question from hiring manager delimited by triple backticks \.


```Question: Kakao Bank, a same but different bank, please describe the reason for the job you applied for based on your differentiated capabilities and skills.``` \.


To address the query
revise candidate Kim JinGu story using triple backticks```Draft: I'm very interested in new technology. 
Recently, I am very interested in large-scale language models, starting with master's thesis. Currently, I am using the license of Meta's Llama2 model, which is distributed free of charge, and understand Prompt Engineering. The cover letter was also able to be written with the great help of Prompt Engineering, and 

In [66]:
ans2 = llm(answer_2)

In [67]:
print(ans2)

  As a highly skilled data analyst, I am eager to apply my expertise in crafting compelling appeals to hiring managers to secure the position of Data Analyst at Kakao Bank. With my proficiency in programming languages such as SQL, Python, and R, and my knowledge of machine learning algorithms and models, including deep learning and natural language processing, I am confident in my ability to analyze large datasets and extract valuable insights.

My experience in developing and applying various data models, such as regression, decision trees, and clustering, has equipped me with the skills necessary to analyze and visualize complex data sets. Additionally, my experience in communicating complex data insights and models to non-technical stakeholders has honed my ability to distill technical information into actionable recommendations.

I am well-versed in collaboration with cross-functional teams, including data scientists, software engineers, and product managers, and have a proven trac